<a href="https://colab.research.google.com/github/jamunozlab/planetary_science_spring_2024/blob/main/mini_project_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kepler exoplanets

Kepler-10b is the first rocky planet detected by the Kepler Space Telescope, a NASA mission launched in March 2009 and deactivated in November 2018. Kepler-10b has been confirmed by the W. M. Keck Telescope in Hawaii. It orbits the star Kepler-10.

In this mini project, you will download and analyze the light intensity measurements from Kepler that led to the detection of Kepler-10b. The packages and the code needed to perform the analysis are provided, but you are asked to qualitatively explain what the analysis methods do to the data. You are also asked to provide a quantitative estimate of the radius of Kepler-10b and explore data for star Kepler-20.

For a quick introduction to Kepler and Kepler-10, watch this [YouTube video](https://www.youtube.com/watch?v=S5ZJEiGOHTo).

In [ ]:
# Evaluate this cell to download and install ligthkurve
# This will be used to access and manipulated Kepler data
# You will see a bunch of messages, worry you must not
# It is only temporarily intalled on your Google Drive (not local machine)
! python -m pip install lightkurve --upgrade

In [4]:
# Import the packages we need for this mini project
import lightkurve as lk
import numpy as np
import matplotlib as plt
import pandas as pd

In [ ]:
# Download the data for star Kepler-10
# The data is stored in a variable called "data"
# Kindly ignore warning messages
data = lk.search_targetpixelfile("Kepler-10", author="Kepler", cadence="long").download()

In [ ]:
# Plot a single frame (snapshot) of kepler-10
# Look at different frames and determine whether there are differences
# What are the units?
data.plot(frame=50, scale='log', show_colorbar=True);

In [ ]:
# The aperture information is part of the data you downloaded
# It is store in the variable "data.pipeline_mask"
# What does the method "to_lightcurve()" do to the data?
lc = data.to_lightcurve(aperture_mask=data.pipeline_mask)
lc.plot();

In [ ]:
# "flatten()" is a method to make the data smoother using a Savitzky–Golay filter
# This increases the precision without distorting the signal
# The red line shows the smoothed out data
# Run this part of the code with different values of the parameter "window_length"
# What qualitative and quantitative differences does it make on your analysis?
flat, trend = lc.flatten(window_length=51, return_trend=True)
ax = lc.errorbar(label="Kepler-10")
trend.plot(ax=ax, color='red', lw=2, label='Trend');

In [ ]:
# The variable "flat" holds the difference between the raw data and the trend
pd.Series(flat.flux, index=flat.time).plot(ylim=(0.9995, 1.0005), ylabel="Flux [e-/s]", xlabel="Time [days]") #.flux(label="Kepler-10");

In [ ]:
# The variable "flat" also holds the error bars
flat.errorbar(label="Kepler-10");

In [ ]:
# "to_periodogram()" is a method to transform time series data into a power spectrum
# What is a power spectrum? Why is this useful to detect transiting exoplanets?
# The Box Least Squares (BLS) method is used to generate the power spectrum
# Describe qualitatively the mechanics and function of BLS
periodogram = flat.to_periodogram(method="bls", period=np.arange(0.4, 1.8, 0.001))
periodogram.plot();

In [ ]:
# The period at the maximum power is stored in "periodogram.period_at_max_power"
# For simplicity, we store this value in the variable "best_fit_period"
# Explain the location of the other peaks in the power spectrum
best_fit_period = periodogram.period_at_max_power
print('Best fit period: {:.3f}'.format(best_fit_period))

In [ ]:
# The "fold()" method folds the data with a given periodicity, in this "case best_fit_period"
# This makes the visualization of the light intensity dip fairly easy
# What happens when you modify the value of the argument "period"?
flat.fold(period=best_fit_period, epoch_time=periodogram.transit_time_at_max_power).errorbar();

The star Kepler-10 has a radius of 1.065 $R_\odot$ (where the solar radius $R_\odot$ = 6.95700 $\times 10^8$ meters).

Based on the normalized flux plot above, what is the radius of planet Kepler-10b? Assume that both Kepler-10 and Kepler-10b are disks.

What is the

Another [YouTube video](https://www.youtube.com/watch?v=8_k3SoNSgYo), this one about the Kepler-10 system. Enjoy!